Multi Linear Band Gap Models based on Perovskite Compositions
=============================================================

**Author:** Panayotis Manganaris



## dependencies



"cmcl" and "yogi" are our in-house modules for analyzing chemical
compositions and enabling different nuts and bolts of ML algorithms.



In [ ]:
# featurization
import cmcl
from cmcl import Categories
# multi-criterion model evaluation
from yogi.model_selection import pandas_validation_curve as pvc
from yogi.metrics.pandas_scoring import PandasScoreAdaptor as PSA
from yogi.metrics.pandas_scoring import batch_score
# visualization convenience
from spyglass.model_imaging import parityplot

The Intel distribution provides accelerated ml algorithms. Run this
cell before importing the algorithms.



In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
# data tools
import pandas as pd
import numpy as np
# feature engineering
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, Normalizer
# predictors
from sklearn.linear_model import LinearRegression, ElasticNet
## pipeline workflow
from sklearn.pipeline import make_pipeline as mkpipe
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV as gsCV
# model eval
from sklearn.base import clone
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, explained_variance_score, max_error
import joblib
#visualization
from sklearn import set_config
import matplotlib.pyplot as plt
import seaborn as sns
# ignore all FutureWarnings -- handling coming in a future version of yogi
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Load Data Using Cmcl and Compute Composition Vectors



## load data



loading stored targets and reference material



In [ ]:
my = pd.read_csv("./mannodi_data.csv").set_index(["index", "Formula", "sim_cell"])
lookup = pd.read_csv("./constituent_properties.csv").set_index("Formula")

## compute features



cmcl provides an "ft" (feature) pandas DataFrame accessor. This
accessor exposes batch feature extraction tools. The function ft.comp
extracts composition vectors from the formula string in a dataframe
(or dataframe index).

The abx function of the collect accessor is a convenience function for
grouping the resulting composition constituents by site membership



In [ ]:
mc = my.ft.comp() # compute numerical compostion vectors from strings
mc = mc.collect.abx() # convenient site groupings for perovskites data

this can be used with the logif tool in categories to quickly
categorize records by their mix status. that status is assigned to the
index of the respective tables for further reference



In [ ]:
mixlog = mc.groupby(level=0, axis=1).count()
mix = mixlog.pipe(Categories.logif, condition=lambda x: x>1, default="pure", catstring="and")
mc = mc.assign(mix=mix).set_index("mix", append=True)
my = my.assign(mix=mix).set_index("mix", append=True)

## Model BG Using Composition Vectors



This model will be based on a simple least squares linear
regression. The data will be normalized as discussed in visualizations.



### Make Composition Pipeline



In [ ]:
fillna = SimpleImputer(strategy="constant", fill_value=0.0)
cpipe = mkpipe(fillna, Normalizer('l1'), LinearRegression())

### Scoring Scheme



#### prepare subset scoring weights and ordinal group labels



In [ ]:
mixweight = pd.get_dummies(mix)
mixcat = pd.Series(OrdinalEncoder().fit_transform(mix.values.reshape(-1, 1)).reshape(-1),
                     index=mc.index).astype(int)

#### Define Scoring Metrics



In [ ]:
site_mse = PSA(mean_squared_error).score
scorings = {'r2': make_scorer(r2_score),
            'ev': make_scorer(explained_variance_score),
            'maxerr': make_scorer(max_error, greater_is_better=False),
            'rmse': make_scorer(mean_squared_error, greater_is_better=False, squared=False),
            'A_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.A),
            'B_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.B),
            'X_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.X),
            'BandX_rmse': make_scorer(site_mse, greater_is_better=False,
                                      squared=False, sample_weight=mixweight.BandX),
            'Pure_rmse': make_scorer(site_mse, greater_is_better=False,
                                     squared=False, sample_weight=mixweight.pure),}

### Make Dedicated Test Train Split



In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=None)
train_idx, test_idx = next(sss.split(mc, mixcat)) #stratify split by mix categories
mc_tr, mc_ts = mc.iloc[train_idx], mc.iloc[test_idx]
my_tr, my_ts = my.iloc[train_idx], my.iloc[test_idx]
mixcat_tr, mixcat_ts = mixcat.iloc[train_idx], mixcat.iloc[test_idx]

### Learning Curves &#x2013; Using Deterministically Random Cross Validation



In [ ]:
kf_lc = KFold(n_splits=10, shuffle=True, random_state=111)

In [ ]:
with joblib.parallel_backend('multiprocessing'):
  LC = pvc(learning_curve, cpipe, mc_tr, my_tr.PBE_bg_eV,
           train_sizes=np.linspace(0.1, 1.0, 10), cv=kf_lc, scoring=scorings)
  LC = LC.melt(id_vars=["partition"], ignore_index=False).reset_index()

In [ ]:
p = sns.FacetGrid(LC, col="score", hue="partition", col_wrap=3, sharey=False)
p.map(sns.lineplot, "train_sizes", "value")
p.add_legend()
p.figure.show()

It appears that 2-3 fold cross-validation is sufficient (training with
200/400 or 260/400 points, validating with the compliment)

In the case of max error, it looks like using any more than about 100 datapoints makes for a pretty good chance of overfitting, but the rest of the metrics do not support this conclusion.

The Linear Regressor validation scores tend to cap out after about
200 data points have been seen. Giving this regressor more data likely
won't help.



### Obtain Generality Baseline



In [ ]:
gkf = GroupKFold(n_splits=4)

In [ ]:
def test_generality(estimator, X_tr, y_tr, groups_tr, X_ts, y_ts, groups_ts):
    estimator = clone(estimator) #unfitted, cloned params
    gentpl = gkf.split(X_tr, y_tr, groups=groups_tr), gkf.split(X_ts, y_ts, groups=groups_ts)
    #train and test index generators, in order
    val_scores = []
    tst_scores = []
    for train_idx, val_idx, _, tst_idx in [sum(gengroup, ()) for gengroup in zip(*gentpl)]:
        tr_val_group_names = groups_tr.iloc[val_idx].index.get_level_values("mix").unique()
        ts_group_names = groups_ts.iloc[tst_idx].index.get_level_values("mix").unique()
        #fit to tr part
        estimator.fit(X_tr.iloc[train_idx], y_tr.iloc[train_idx])
        #get val and test scores
        tr_val_score_series = pd.Series(batch_score(estimator, X_tr.iloc[val_idx], y_tr.iloc[val_idx], **scorings))
        tr_val_score_series.name="_&_".join(tr_val_group_names)
        ts_score_series = pd.Series(batch_score(estimator, X_ts.iloc[tst_idx], y_ts.iloc[tst_idx], **scorings))
        ts_score_series.name="_&_".join(ts_group_names)
        val_scores.append(tr_val_score_series)
        tst_scores.append(ts_score_series)
    tr_val_scores = pd.concat(val_scores, axis=1).assign(partition="validation")
    ts_scores = pd.concat(tst_scores, axis=1).assign(partition="test")
    group_scores = pd.concat([tr_val_scores, ts_scores]).round(5).drop_duplicates(keep="first")
    return group_scores

In [ ]:
test_generality(cpipe, mc_tr, my_tr.PBE_bg_eV, mixcat_tr, mc_ts, my_ts.PBE_bg_eV, mixcat_ts)

-   extrapolations are not very accurate, but generally within 0.5 eV

### Best Model



#### Parametrize



Default parameters used. Linear regression is very simple



#### Train Final Estimator



In [ ]:
#cpipe = cpipe.set_params(**{k:v[0] for k,v in grid[0].items()})
cpipe.fit(mc_tr, my_tr.PBE_bg_eV)

#### evaluate



In [ ]:
#change between tr and ts suffixes to see test vs train pairity plot
p, data = parityplot(cpipe, mc_tr, my_tr.PBE_bg_eV.to_frame(), aspect=1.0, hue="mix")
p.figure.show()

In [ ]:
#change between tr and ts suffixes to see test vs train scores -- both are good
pd.Series(batch_score(cpipe, mc_ts, my_ts.PBE_bg_eV, **scorings)).to_frame()

#### discussion



The linear model is actually pretty reasonable, but it's unlikely that
it's learning anything very fundamental about the underlying physics
of the Pervoskite system. however, that doesn't mean it reveals nothing.



In [ ]:
interpret = pd.Series(cpipe[-1].coef_, index=mc_tr.columns)
interpret

In [ ]:
interpret.groupby(level=0).aggregate(lambda x: np.sqrt(sum(x**2))) #compute the root mean square model coefficient per site feature

To illustrate, the coefficients that define the linear combination do
not find much use in X-site elements. however, the B site elements
contribute much more to the band gap on average, which is consistent
with our physical understanding.

A site elements also prove to be relevant.

Likewise, with respect to the generality measure conducted earlier, it
seems the presence of individual elements is far more predictive of
the total band gap than mix status. This would explain why X-site and
A-site alloys are sufficient to predict the band gaps of B-site alloys.
Despite those groups containing no B-site alloys themselves, they do
contain a representative sample of B-site elements.



## Model BG Using Composition Vectors



The features used here, especially after l1 normalization, are
proportions of a total. By definition they are correlated. So, it is
worth using a more intelligent linear regression to try extracting an
indication of the most impactful features.

ElasticNet uses Lasso in combination with l1 and l2 regularization of
the model parameters to encourage sparsity. however, it is not
especially strict. unlike Lasso (another sparsifying linear
regressor), ElasticNet will use multiple sets of correlated features
instead of &#x2013; effectively randomly &#x2013; picking only one in the effort
to return sparse coefficients.



### Make Composition Pipeline



The higher the alpha value, the more aggressively elasticnet will
minimize it's weights. an alpha value of 1 will force all weights to
zero.



In [ ]:
fillna = SimpleImputer(strategy="constant", fill_value=0.0)
cpipe = mkpipe(fillna, Normalizer('l1'), ElasticNet(alpha=0.001, l1_ratio=0.5))

### Make Dedicated Test Train Split



In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=None)
train_idx, test_idx = next(sss.split(mc, mixcat)) #stratify split by mix categories
mc_tr, mc_ts = mc.iloc[train_idx], mc.iloc[test_idx]
my_tr, my_ts = my.iloc[train_idx], my.iloc[test_idx]
mixcat_tr, mixcat_ts = mixcat.iloc[train_idx], mixcat.iloc[test_idx]

### Learning Curves &#x2013; Using Deterministically Random Cross Validation



In [ ]:
kf_lc = KFold(n_splits=10, shuffle=True, random_state=111)

In [ ]:
with joblib.parallel_backend('multiprocessing'):
  LC = pvc(learning_curve, cpipe, mc_tr, my_tr.PBE_bg_eV,
           train_sizes=np.linspace(0.1, 1.0, 10), cv=kf_lc, scoring=scorings)
  LC = LC.melt(id_vars=["partition"], ignore_index=False).reset_index()

In [ ]:
p = sns.FacetGrid(LC, col="score", hue="partition", col_wrap=3, sharey=False)
p.map(sns.lineplot, "train_sizes", "value")
p.add_legend()
p.figure.show()

Like the normal least squares regressor, elastic net quickly saturates to a maximum score.

-   scores are decent
-   the point really is to see if any of the features are effectively redundant contributors.



### Obtain Generality Baseline



In [ ]:
gkf = GroupKFold(n_splits=4)

In [ ]:
test_generality(cpipe, mc_tr, my_tr.PBE_bg_eV, mixcat_tr, mc_ts, my_ts.PBE_bg_eV, mixcat_ts)

-   extrapolations are not very accurate, but generally within 0.5 eV
-   scores are overall a little bit worse than the normal OLS regressor

### Best Model



#### Parametrize



Default parameters used. Linear regression is very simple



#### Train Final Estimator



In [ ]:
#cpipe = cpipe.set_params(**{k:v[0] for k,v in grid[0].items()})
cpipe.fit(mc_tr, my_tr.PBE_bg_eV)

#### evaluate



In [ ]:
#change between tr and ts suffixes to see test vs train pairity plot
p, data = parityplot(cpipe, mc_tr, my_tr.PBE_bg_eV.to_frame(), aspect=1.0, hue="mix")
p.figure.show()

In [ ]:
#change between tr and ts suffixes to see test vs train scores -- both are good
pd.Series(batch_score(cpipe, mc_ts, my_ts.PBE_bg_eV, **scorings)).to_frame()

#### discussion



The linear model is actually pretty reasonable, but it's unlikely that
it's learning anything very fundamental about the underlying physics
of the Pervoskite system. however, that doesn't mean it reveals nothing.



In [ ]:
interpret = pd.Series(cpipe[-1].coef_, index=mc_tr.columns)
interpret

Here we start to see that more complicated models appear to take
A-site elements more seriously.



In [ ]:
interpret.groupby(level=0).aggregate(lambda x: np.sqrt(sum(x**2)))

The relevance of A site elements seems to fluctuate depending on the random splitting -- try the splitting and final model training repeatedly to see the variation.

This exercise results in one important conclusion:

Despite the fact that the composition space definitely consists of
correlated features, very few of them can be easily called
redundant.

Additionally, comparing the ElasticNet scores to the standard linear
regression shows using all features seems to result in better
performance.

